In [14]:
"""
Polymarket Data-API 完整实现
基于官方文档: https://docs.polymarket.com/api-reference/data-api
"""

import requests
import json
from typing import Optional, List, Dict, Any
from datetime import datetime

# Data-API 基础 URL
BASE_URL = "https://data-api.polymarket.com"

class PolymarketDataAPI:
    """Polymarket Data-API 客户端"""
    
    def __init__(self, base_url: str = BASE_URL):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        })
    
    def _get(self, endpoint: str, params: Optional[Dict] = None) -> Dict[str, Any]:
        """发送 GET 请求"""
        url = f"{self.base_url}{endpoint}"
        try:
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"请求失败: {e}")
            if hasattr(e.response, 'text'):
                print(f"响应内容: {e.response.text}")
            raise
    
    # ==================== Health ====================
    
    def health_check(self) -> Dict[str, Any]:
        """
        健康检查端点
        GET /health
        """
        return self._get("/health")
    
    # ==================== Core Endpoints ====================
    
    def get_positions(
        self,
        user: str,
        market: Optional[List[str]] = None,
        event_id: Optional[List[int]] = None,
        size_threshold: float = 1.0,
        redeemable: bool = False,
        mergeable: bool = False,
        limit: int = 100,
        offset: int = 0,
        sort_by: str = "TOKENS",
        sort_direction: str = "DESC",
        title: Optional[str] = None
    ) -> List[Dict[str, Any]]:
        """
        获取用户当前持仓
        GET /positions
        
        参数:
            user: 用户地址 (0x开头的40位十六进制字符串，必需)
            market: 条件ID列表 (逗号分隔，与event_id互斥)
            event_id: 事件ID列表 (逗号分隔，与market互斥)
            size_threshold: 持仓大小阈值 (默认: 1.0)
            redeemable: 是否可赎回 (默认: False)
            mergeable: 是否可合并 (默认: False)
            limit: 返回数量限制 (默认: 100, 范围: 0-500)
            offset: 偏移量 (默认: 0, 范围: 0-10000)
            sort_by: 排序字段 (CURRENT, INITIAL, TOKENS, CASHPNL, PERCENTPNL, TITLE, RESOLVING, PRICE, AVGPRICE)
            sort_direction: 排序方向 (ASC, DESC)
            title: 标题搜索 (最大长度: 100)
        """
        params = {
            "user": user,
            "sizeThreshold": size_threshold,
            "redeemable": redeemable,
            "mergeable": mergeable,
            "limit": limit,
            "offset": offset,
            "sortBy": sort_by,
            "sortDirection": sort_direction
        }
        
        if market:
            params["market"] = ",".join(market)
        if event_id:
            params["eventId"] = ",".join(map(str, event_id))
        if title:
            params["title"] = title
        
        return self._get("/positions", params=params)
    
    def get_trades(
        self,
        user: Optional[str] = None,
        market: Optional[List[str]] = None,
        event_id: Optional[List[int]] = None,
        limit: int = 100,
        offset: int = 0,
        sort_by: str = "TIMESTAMP",
        sort_direction: str = "DESC"
    ) -> List[Dict[str, Any]]:
        """
        获取用户或市场的交易记录
        GET /trades
        
        参数:
            user: 用户地址 (可选)
            market: 条件ID列表 (可选)
            event_id: 事件ID列表 (可选)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
            sort_by: 排序字段 (默认: TIMESTAMP)
            sort_direction: 排序方向 (ASC, DESC)
        """
        params = {
            "limit": limit,
            "offset": offset,
            "sortBy": sort_by,
            "sortDirection": sort_direction
        }
        
        if user:
            params["user"] = user
        if market:
            params["market"] = ",".join(market)
        if event_id:
            params["eventId"] = ",".join(map(str, event_id))
        
        return self._get("/trades", params=params)
    
    def get_user_activity(
        self,
        user: str,
        limit: int = 100,
        offset: int = 0,
        sort_by: str = "TIMESTAMP",
        sort_direction: str = "DESC"
    ) -> List[Dict[str, Any]]:
        """
        获取用户活动
        GET /activity
        
        参数:
            user: 用户地址 (必需)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
            sort_by: 排序字段 (默认: TIMESTAMP)
            sort_direction: 排序方向 (ASC, DESC)
        """
        params = {
            "user": user,
            "limit": limit,
            "offset": offset,
            "sortBy": sort_by,
            "sortDirection": sort_direction
        }
        
        return self._get("/activity", params=params)
    
    def get_all_user_activity(
        self,
        user: str,
        sort_by: str = "TIMESTAMP",
        sort_direction: str = "DESC",
        batch_size: int = 100,
        max_records: Optional[int] = None,
        verbose: bool = True
    ) -> List[Dict[str, Any]]:
        """
        循环获取用户的所有历史活动记录
        
        参数:
            user: 用户地址 (必需)
            sort_by: 排序字段 (默认: TIMESTAMP)
            sort_direction: 排序方向 (ASC, DESC)
            batch_size: 每批获取的记录数 (默认: 100, 最大: 500)
            max_records: 最大获取记录数 (None 表示获取所有记录)
            verbose: 是否显示进度信息 (默认: True)
        
        返回:
            所有活动记录的列表
        """
        all_activities = []
        offset = 0
        batch_size = min(batch_size, 500)  # API 限制最大 500
        
        if verbose:
            print(f"开始获取用户 {user} 的所有活动记录...")
        
        while True:
            try:
                if verbose:
                    print(f"正在获取第 {offset // batch_size + 1} 批 (offset: {offset}, limit: {batch_size})...")
                
                # 获取当前批次
                batch = self.get_user_activity(
                    user=user,
                    limit=batch_size,
                    offset=offset,
                    sort_by=sort_by,
                    sort_direction=sort_direction
                )
                
                # 如果没有数据，说明已经获取完所有记录
                if not batch or len(batch) == 0:
                    if verbose:
                        print("已获取所有记录")
                    break
                
                # 添加到总列表
                all_activities.extend(batch)
                
                if verbose:
                    print(f"  已获取 {len(batch)} 条记录，累计: {len(all_activities)} 条")
                
                # 如果返回的记录数少于 batch_size，说明已经是最后一批
                if len(batch) < batch_size:
                    if verbose:
                        print("已获取所有记录（最后一批）")
                    break
                
                # 如果设置了最大记录数限制
                if max_records and len(all_activities) >= max_records:
                    all_activities = all_activities[:max_records]
                    if verbose:
                        print(f"已达到最大记录数限制: {max_records}")
                    break
                
                # 更新偏移量，准备获取下一批
                offset += batch_size
                
                # 安全限制：防止无限循环（API 限制 offset 最大 10000）
                if offset > 10000:
                    if verbose:
                        print(f"警告: 已达到 API 偏移量限制 (10000)，停止获取")
                    break
                
            except Exception as e:
                print(f"获取数据时出错: {e}")
                break
        
        if verbose:
            print(f"\n总共获取了 {len(all_activities)} 条活动记录")
        
        return all_activities
    
    def get_top_holders(
        self,
        market: Optional[str] = None,
        event_id: Optional[int] = None,
        limit: int = 100,
        offset: int = 0
    ) -> List[Dict[str, Any]]:
        """
        获取市场顶级持有者
        GET /top-holders
        
        参数:
            market: 条件ID (与event_id互斥)
            event_id: 事件ID (与market互斥)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
        """
        params = {
            "limit": limit,
            "offset": offset
        }
        
        if market:
            params["market"] = market
        if event_id:
            params["eventId"] = event_id
        
        return self._get("/top-holders", params=params)
    
    def get_total_value(
        self,
        user: str,
        market: Optional[List[str]] = None,
        event_id: Optional[List[int]] = None
    ) -> Dict[str, Any]:
        """
        获取用户持仓总价值
        GET /total-value
        
        参数:
            user: 用户地址 (必需)
            market: 条件ID列表 (可选)
            event_id: 事件ID列表 (可选)
        """
        params = {"user": user}
        
        if market:
            params["market"] = ",".join(market)
        if event_id:
            params["eventId"] = ",".join(map(str, event_id))
        
        return self._get("/total-value", params=params)
    
    def get_closed_positions(
        self,
        user: str,
        market: Optional[List[str]] = None,
        event_id: Optional[List[int]] = None,
        limit: int = 100,
        offset: int = 0,
        sort_by: str = "TIMESTAMP",
        sort_direction: str = "DESC"
    ) -> List[Dict[str, Any]]:
        """
        获取用户已关闭的持仓
        GET /closed-positions
        
        参数:
            user: 用户地址 (必需)
            market: 条件ID列表 (可选)
            event_id: 事件ID列表 (可选)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
            sort_by: 排序字段 (默认: TIMESTAMP)
            sort_direction: 排序方向 (ASC, DESC)
        """
        params = {
            "user": user,
            "limit": limit,
            "offset": offset,
            "sortBy": sort_by,
            "sortDirection": sort_direction
        }
        
        if market:
            params["market"] = ",".join(market)
        if event_id:
            params["eventId"] = ",".join(map(str, event_id))
        
        return self._get("/closed-positions", params=params)
    
    def get_trader_leaderboard(
        self,
        period: str = "ALL_TIME",
        limit: int = 100,
        offset: int = 0
    ) -> List[Dict[str, Any]]:
        """
        获取交易员排行榜
        GET /trader-leaderboard
        
        参数:
            period: 时间周期 (ALL_TIME, WEEK, MONTH, YEAR)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
        """
        params = {
            "period": period,
            "limit": limit,
            "offset": offset
        }
        
        return self._get("/trader-leaderboard", params=params)


# ==================== 使用示例 ====================

# 创建 API 客户端实例
api = PolymarketDataAPI()

print("Polymarket Data-API 客户端已创建")
print(f"基础 URL: {api.base_url}")
print("\n可用方法:")
print("- health_check() - 健康检查")
print("- get_positions(user, ...) - 获取用户当前持仓")
print("- get_trades(user=None, market=None, ...) - 获取交易记录")
print("- get_user_activity(user, ...) - 获取用户活动")
print("- get_all_user_activity(user, ...) - 循环获取所有历史活动记录")
print("- get_top_holders(market=None, event_id=None, ...) - 获取顶级持有者")
print("- get_total_value(user, ...) - 获取持仓总价值")
print("- get_closed_positions(user, ...) - 获取已关闭持仓")
print("- get_trader_leaderboard(period='ALL_TIME', ...) - 获取交易员排行榜")


Polymarket Data-API 客户端已创建
基础 URL: https://data-api.polymarket.com

可用方法:
- health_check() - 健康检查
- get_positions(user, ...) - 获取用户当前持仓
- get_trades(user=None, market=None, ...) - 获取交易记录
- get_user_activity(user, ...) - 获取用户活动
- get_all_user_activity(user, ...) - 循环获取所有历史活动记录
- get_top_holders(market=None, event_id=None, ...) - 获取顶级持有者
- get_total_value(user, ...) - 获取持仓总价值
- get_closed_positions(user, ...) - 获取已关闭持仓
- get_trader_leaderboard(period='ALL_TIME', ...) - 获取交易员排行榜


In [19]:
import requests
BASE_URL = "https://data-api.polymarket.com"
def get_user_activity(
        user: str,
        limit: int = 100,
        offset: int = 0,
        sort_by: str = "TIMESTAMP",
        sort_direction: str = "DESC"
    ) -> List[Dict[str, Any]]:
        """
        获取用户活动
        GET /activity
        
        参数:
            user: 用户地址 (必需)
            limit: 返回数量限制 (默认: 100)
            offset: 偏移量 (默认: 0)
            sort_by: 排序字段 (默认: TIMESTAMP)
            sort_direction: 排序方向 (ASC, DESC)
        """
        params = {
            "user": user,
            "limit": limit,
            "offset": offset,
            "sortBy": sort_by,
            "sortDirection": sort_direction
        }
        
        return requests.get(BASE_URL+"/activity", params=params)
res = get_user_activity("0x45deaaD70997b2998FBb9433B1819178e34B409C")

In [20]:
res.json()

[{'proxyWallet': '0x45deaad70997b2998fbb9433b1819178e34b409c',
  'timestamp': 1766471661,
  'conditionId': '0x880d5c2aab3fd10c0e18efbf62e0fa724472473e244e08b11d9b73b89d21b52c',
  'type': 'REDEEM',
  'size': 100,
  'usdcSize': 100,
  'transactionHash': '0x99cfeb680bc13ae0d1093cbf882f1d4337a908b588edde82f98a95700102ab0d',
  'price': 0,
  'asset': '',
  'side': '',
  'outcomeIndex': 999,
  'title': 'Will Drake release a new song in 2025?',
  'slug': 'will-drake-release-a-new-song-in-2025',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/which-artists-will-release-new-albums-in-2025-AfUvGA90JtU_.jpg',
  'eventSlug': 'which-artists-will-release-a-new-song-before-2026',
  'outcome': '',
  'name': 'duchanghui',
  'pseudonym': 'Outstanding-Clogs',
  'bio': '',
  'profileImage': '',
  'profileImageOptimized': ''},
 {'proxyWallet': '0x45deaad70997b2998fbb9433b1819178e34b409c',
  'timestamp': 1766470943,
  'conditionId': '0xe5e828881862659c7c8922c0ea7d800178347e1d267739a4cf1632bb5

In [3]:
# ==================== 示例 1: 健康检查 ====================
print("示例 1: 健康检查")
print("-" * 50)
try:
    health = api.health_check()
    print(json.dumps(health, indent=2, ensure_ascii=False))
except Exception as e:
    print(f"错误: {e}")


示例 1: 健康检查
--------------------------------------------------
请求失败: 404 Client Error: Not Found for url: https://data-api.polymarket.com/health
响应内容: 404 page not found

错误: 404 Client Error: Not Found for url: https://data-api.polymarket.com/health


In [5]:
# ==================== 示例 2: 获取用户当前持仓 ====================
# 注意: 需要替换为实际的用户地址
print("示例 2: 获取用户当前持仓")
print("-" * 50)

# 示例用户地址 (请替换为实际地址)
example_user = "0x45deaaD70997b2998FBb9433B1819178e34B409C"

try:
    positions = api.get_positions(
        user=example_user,
        limit=10,  # 只获取前10条
        sort_by="CURRENT",  # 按当前价值排序
        sort_direction="DESC"
    )
    print(f"找到 {len(positions)} 个持仓")
    if positions:
        print("\n前3个持仓:")
        for i, pos in enumerate(positions[:3], 1):
            print(f"\n{i}. {pos.get('title', 'N/A')}")
            print(f"   当前价值: ${pos.get('currentValue', 0):.2f}")
            print(f"   盈亏: ${pos.get('cashPnl', 0):.2f} ({pos.get('percentPnl', 0):.2f}%)")
            print(f"   持仓数量: {pos.get('size', 0)}")
except Exception as e:
    print(f"错误: {e}")
    print("提示: 请确保用户地址有效，或该用户有持仓记录")


示例 2: 获取用户当前持仓
--------------------------------------------------
找到 9 个持仓

前3个持仓:

1. Will Drake release a new song in 2025?
   当前价值: $99.95
   盈亏: $0.06 (0.06%)
   持仓数量: 100

2. Jarrett Allen: Points Over 14.5
   当前价值: $99.85
   盈亏: $-0.05 (-0.05%)
   持仓数量: 100

3. Donovan Mitchell: Points Over 30.5
   当前价值: $99.85
   盈亏: $-0.05 (-0.05%)
   持仓数量: 100


In [6]:
# ==================== 示例 3: 获取交易记录 ====================
print("示例 3: 获取交易记录")
print("-" * 50)

try:
    # 获取特定用户的交易记录
    trades = api.get_trades(
        user=example_user,
        limit=10,
        sort_by="TIMESTAMP",
        sort_direction="DESC"
    )
    print(f"找到 {len(trades)} 笔交易")
    if trades:
        print("\n最近3笔交易:")
        for i, trade in enumerate(trades[:3], 1):
            print(f"\n{i}. 交易ID: {trade.get('id', 'N/A')}")
            print(f"   时间: {trade.get('timestamp', 'N/A')}")
            print(f"   金额: ${trade.get('amount', 0):.2f}")
except Exception as e:
    print(f"错误: {e}")


示例 3: 获取交易记录
--------------------------------------------------
找到 10 笔交易

最近3笔交易:

1. 交易ID: N/A
   时间: 1766470943
   金额: $0.00

2. 交易ID: N/A
   时间: 1766469275
   金额: $0.00

3. 交易ID: N/A
   时间: 1766469263
   金额: $0.00


In [13]:
# ==================== 示例 4: 获取用户活动 ====================
print("示例 4: 获取用户活动")
print("-" * 50)

try:
    activities = api.get_user_activity(
        user=example_user,
        limit=10000
    )
    print(f"找到 {len(activities)} 条活动记录")
    if activities:
        print("\n最近3条活动:")
        for i, activity in enumerate(activities[:3], 1):
            print(f"\n{i}. {json.dumps(activity, indent=2, ensure_ascii=False)}")
except Exception as e:
    print(f"错误: {e}")


示例 4: 获取用户活动
--------------------------------------------------
请求失败: HTTPSConnectionPool(host='data-api.polymarket.com', port=443): Max retries exceeded with url: /activity?user=0x45deaaD70997b2998FBb9433B1819178e34B409C&limit=10000&offset=0&sortBy=TIMESTAMP&sortDirection=DESC (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
错误: HTTPSConnectionPool(host='data-api.polymarket.com', port=443): Max retries exceeded with url: /activity?user=0x45deaaD70997b2998FBb9433B1819178e34B409C&limit=10000&offset=0&sortBy=TIMESTAMP&sortDirection=DESC (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


In [15]:
# ==================== 示例 4.5: 循环获取所有用户活动记录 ====================
print("示例 4.5: 循环获取所有用户活动记录")
print("-" * 50)

try:
    # 使用 get_all_user_activity 获取所有历史记录
    all_activities = api.get_all_user_activity(
        user=example_user,
        batch_size=100,  # 每批获取100条
        verbose=True  # 显示进度信息
    )
    
    print(f"\n总共获取了 {len(all_activities)} 条活动记录")
    
    if all_activities:
        print("\n最早和最晚的活动记录:")
        print(f"最早: {all_activities[-1].get('timestamp', 'N/A')}")
        print(f"最晚: {all_activities[0].get('timestamp', 'N/A')}")
        
        # 按类型统计
        activity_types = {}
        for activity in all_activities:
            activity_type = activity.get('type', 'UNKNOWN')
            activity_types[activity_type] = activity_types.get(activity_type, 0) + 1
        
        print("\n活动类型统计:")
        for activity_type, count in sorted(activity_types.items(), key=lambda x: x[1], reverse=True):
            print(f"  {activity_type}: {count} 条")
        
        # 显示前3条记录
        print("\n前3条活动记录:")
        for i, activity in enumerate(all_activities[:3], 1):
            print(f"\n{i}. 类型: {activity.get('type', 'N/A')}")
            print(f"   时间戳: {activity.get('timestamp', 'N/A')}")
            print(f"   详情: {json.dumps(activity, indent=2, ensure_ascii=False)}")
    
except Exception as e:
    print(f"错误: {e}")
    import traceback
    traceback.print_exc()


示例 4.5: 循环获取所有用户活动记录
--------------------------------------------------
开始获取用户 0x45deaaD70997b2998FBb9433B1819178e34B409C 的所有活动记录...
正在获取第 1 批 (offset: 0, limit: 100)...
  已获取 100 条记录，累计: 100 条
正在获取第 2 批 (offset: 100, limit: 100)...
  已获取 100 条记录，累计: 200 条
正在获取第 3 批 (offset: 200, limit: 100)...
  已获取 100 条记录，累计: 300 条
正在获取第 4 批 (offset: 300, limit: 100)...
  已获取 100 条记录，累计: 400 条
正在获取第 5 批 (offset: 400, limit: 100)...
  已获取 100 条记录，累计: 500 条
正在获取第 6 批 (offset: 500, limit: 100)...
  已获取 100 条记录，累计: 600 条
正在获取第 7 批 (offset: 600, limit: 100)...
  已获取 100 条记录，累计: 700 条
正在获取第 8 批 (offset: 700, limit: 100)...
  已获取 100 条记录，累计: 800 条
正在获取第 9 批 (offset: 800, limit: 100)...
  已获取 100 条记录，累计: 900 条
正在获取第 10 批 (offset: 900, limit: 100)...
  已获取 100 条记录，累计: 1000 条
正在获取第 11 批 (offset: 1000, limit: 100)...
  已获取 100 条记录，累计: 1100 条
正在获取第 12 批 (offset: 1100, limit: 100)...
  已获取 100 条记录，累计: 1200 条
正在获取第 13 批 (offset: 1200, limit: 100)...
  已获取 100 条记录，累计: 1300 条
正在获取第 14 批 (offset: 1300, limit: 100)..

KeyboardInterrupt: 

In [9]:
# ==================== 示例 5: 获取持仓总价值 ====================
print("示例 5: 获取持仓总价值")
print("-" * 50)

try:
    total_value = api.get_total_value(user=example_user)
    print("持仓总价值:")
    print(json.dumps(total_value, indent=2, ensure_ascii=False))
except Exception as e:
    print(f"错误: {e}")


示例 5: 获取持仓总价值
--------------------------------------------------
请求失败: 404 Client Error: Not Found for url: https://data-api.polymarket.com/total-value?user=0x45deaaD70997b2998FBb9433B1819178e34B409C
响应内容: 404 page not found

错误: 404 Client Error: Not Found for url: https://data-api.polymarket.com/total-value?user=0x45deaaD70997b2998FBb9433B1819178e34B409C


In [10]:
# ==================== 示例 6: 获取已关闭的持仓 ====================
print("示例 6: 获取已关闭的持仓")
print("-" * 50)

try:
    closed_positions = api.get_closed_positions(
        user=example_user,
        limit=10
    )
    print(f"找到 {len(closed_positions)} 个已关闭的持仓")
    if closed_positions:
        print("\n前3个已关闭持仓:")
        for i, pos in enumerate(closed_positions[:3], 1):
            print(f"\n{i}. {pos.get('title', 'N/A')}")
            print(f"   已实现盈亏: ${pos.get('realizedPnl', 0):.2f}")
except Exception as e:
    print(f"错误: {e}")


示例 6: 获取已关闭的持仓
--------------------------------------------------
找到 10 个已关闭的持仓

前3个已关闭持仓:

1. Will Drake release a new song in 2025?
   已实现盈亏: $0.11

2. 49ers Team Total: O/U 35.5
   已实现盈亏: $0.10

3. Grizzlies vs. Thunder: 1H O/U 118.5
   已实现盈亏: $0.10


In [ ]:
# ==================== 示例 7: 获取交易员排行榜 ====================
print("示例 7: 获取交易员排行榜")
print("-" * 50)

try:
    leaderboard = api.get_trader_leaderboard(
        period="ALL_TIME",  # 可选: ALL_TIME, WEEK, MONTH, YEAR
        limit=10
    )
    print(f"排行榜前 {len(leaderboard)} 名:")
    for i, trader in enumerate(leaderboard[:5], 1):
        print(f"\n{i}. 用户: {trader.get('user', 'N/A')}")
        print(f"   排名: {trader.get('rank', 'N/A')}")
        print(f"   数据: {json.dumps(trader, indent=2, ensure_ascii=False)}")
except Exception as e:
    print(f"错误: {e}")


In [ ]:
# ==================== 示例 8: 获取市场顶级持有者 ====================
print("示例 8: 获取市场顶级持有者")
print("-" * 50)

# 注意: 需要提供 market (条件ID) 或 event_id
# 这里使用 event_id 作为示例
try:
    # 示例: 使用 event_id (请替换为实际的事件ID)
    top_holders = api.get_top_holders(
        event_id=12345,  # 请替换为实际的事件ID
        limit=10
    )
    print(f"找到 {len(top_holders)} 个顶级持有者")
    if top_holders:
        print("\n前3个持有者:")
        for i, holder in enumerate(top_holders[:3], 1):
            print(f"\n{i}. {json.dumps(holder, indent=2, ensure_ascii=False)}")
except Exception as e:
    print(f"错误: {e}")
    print("提示: 请提供有效的 market 或 event_id 参数")


## 完整 API 方法列表

### Health
- `health_check()` - 健康检查端点

### Core Endpoints
- `get_positions(user, ...)` - 获取用户当前持仓
  - 必需参数: `user` (用户地址)
  - 可选参数: `market`, `event_id`, `size_threshold`, `redeemable`, `mergeable`, `limit`, `offset`, `sort_by`, `sort_direction`, `title`

- `get_trades(user=None, market=None, ...)` - 获取用户或市场的交易记录
  - 可选参数: `user`, `market`, `event_id`, `limit`, `offset`, `sort_by`, `sort_direction`

- `get_user_activity(user, ...)` - 获取用户活动
  - 必需参数: `user` (用户地址)
  - 可选参数: `limit`, `offset`, `sort_by`, `sort_direction`

- `get_top_holders(market=None, event_id=None, ...)` - 获取市场顶级持有者
  - 可选参数: `market` 或 `event_id` (至少一个), `limit`, `offset`

- `get_total_value(user, ...)` - 获取用户持仓总价值
  - 必需参数: `user` (用户地址)
  - 可选参数: `market`, `event_id`

- `get_closed_positions(user, ...)` - 获取用户已关闭的持仓
  - 必需参数: `user` (用户地址)
  - 可选参数: `market`, `event_id`, `limit`, `offset`, `sort_by`, `sort_direction`

- `get_trader_leaderboard(period='ALL_TIME', ...)` - 获取交易员排行榜
  - 可选参数: `period` (ALL_TIME, WEEK, MONTH, YEAR), `limit`, `offset`

## 注意事项

1. **用户地址格式**: 必须是 0x 开头的 42 位十六进制字符串
2. **参数互斥**: `market` 和 `event_id` 在某些端点中互斥，不能同时使用
3. **分页**: 使用 `limit` 和 `offset` 参数进行分页查询
4. **错误处理**: 所有方法都会抛出异常，建议使用 try-except 进行错误处理
5. **API 限制**: 注意 API 的速率限制，避免频繁请求

## 参考文档

- [Polymarket Data-API 文档](https://docs.polymarket.com/api-reference/data-api)
- [获取当前持仓端点](https://docs.polymarket.com/api-reference/core/get-current-positions-for-a-user)
